# NHW 2017. Two common pitfalls in designing of validation pipelines
Mikhail Belyaev

## Feature selection 

### There are three main groups of feature selection methods:
 - filters
 - wrappers
 - embedded methods 

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
% pylab inline

## Filters

- estimate an importance score for each feature
- select K most important one
- there are a lot of different ways to calculate feature importances
- Example: http://scikit-learn.org/stable/modules/feature_selection.html#univariate-feature-selection 

### Example 1. Good classification performance, but low statistical score

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

Generate a simple dataset

In [ ]:
def get_dataset1():
    X = np.random.rand(200, 2)
    X[:100, 0] += 1
    X[100:150, 0] += 2
    X[100:, 1] += 0.1
    y = np.concatenate([np.zeros(100), np.ones(100)])
    return X, y

X, y = get_dataset1()

In [ ]:
def plot_data(X, y):
    x_cols = ['x{}'.format(i) for i in range(X.shape[1])]
    df = pd.DataFrame(np.hstack((X, y[:, np.newaxis])), columns=x_cols+['y'])
    if len(x_cols) == 2:
        sns.pairplot(df, hue='y', x_vars='x0', y_vars='x1', size=6)
    else:
        sns.pairplot(df, hue='y', vars=x_cols)
plot_data(X, y)

In [ ]:
# ANOVA
selector = SelectKBest(f_classif, 1)
selector.fit(X, y)
print(selector.scores_)

In [ ]:
# mutial information
selector = SelectKBest(mutual_info_classif, 1)
selector.fit(X, y)
print(selector.scores_)

### Example 2. Univariate stats doesn't catch bivariate dependencies

In [ ]:
def get_dataset2(shift=0.2):
    X = np.random.rand(1000, 3)
    X = X[np.abs(X[:, 1] - X[:, 0]) < 0.22]
    X = X[np.abs(X[:, 1] - X[:, 0]) > 0.02]
    y = X[:, 1] > X[:, 0] 
    X[y, 2] += shift
    return X, y
X, y = get_dataset2()

In [ ]:
plot_data(X, y)

In [ ]:
# Both univariate methods fail
print(SelectKBest(f_classif, 1).fit(X, y).scores_)
print(SelectKBest(mutual_info_classif, 1).fit(X, y).scores_)

##  Wrappers
 - a greedy alrogithm for feature adding and/or deletion
 - there are a lot of different stratigies (starting points, criteria, etc)
 - an example http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination 

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

svc = SVC(kernel="linear")
rfe = RFE(estimator=svc, n_features_to_select=2, step=1)

In [ ]:
rfe.fit(X, y)
print(rfe.ranking_)

## Embedded 
- features are selected automatically as a part of the learning process 
- an example - linear models with the L1 regularization

<div style="width:60%; text-align:center">
<img src=https://1.bp.blogspot.com/-tXq6Nl2lcNg/V3qzttiZ4sI/AAAAAAAAN_M/6nmjgwydWJUy5Kqt9gFg2Nb12BCTcD4ogCLcB/s1600/LASSO.png>
</div>

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l1', C=0.1)
clf.fit(X, y)
print(clf.coef_)

L1-penalty based approaches are a cool class of methods, but in case of correlated variables it can drop relevant features

Let us make the last component fully irrelevant and try a L1-based method again

In [ ]:
X, y = get_dataset2(shift=0)
df = pd.DataFrame(np.hstack((X, y[:, np.newaxis])), columns=['x0', 'x1', 'x2', 'y'])
sns.pairplot(df, hue='y', vars=['x0', 'x1', 'x2'])

In [ ]:
clf = LogisticRegression(penalty='l1', C=0.1)
clf.fit(X, y)
print(clf.coef_)

### Feature selection: a small example

#### TODO: 
 - load an anonimized dataset from dataset3.csv
 - estimate the 10 most important features (using f_classif)
 - perform cross-validation & estimate classification quality

In [ ]:
def get_dataset3():
    data = pd.read_csv('dataset3.csv')
    y = data['y']
    X = data.drop('y', axis=1)
    return X, y
X, y = get_dataset3()

In [ ]:
print('Data shape is {}'.format(X.shape))
print('Class 0 size is {}'.format(sum(y==0)))
print('Class 1 size is {}'.format(sum(y==1)))

In [ ]:
X.head(5)

#### The number of features is too large, so it seems to be a good idea to start with feature selection

In [ ]:
n_features = 50
selector = SelectKBest(f_classif, n_features)
X_reduced = selector.fit_transform(X, y)
print('The new shape is {}'.format(X_reduced.shape))

#### Now we have a reasonable number of features and can estimate classification accuracy 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV()
scores = cross_val_score(clf, X_reduced, y, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

In [ ]:
clf = LogisticRegressionCV()
scores = cross_val_score(clf, X, y, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))
print('Accuracy of a naive classifier is {}'.format((y==0).mean()))

So, we've obtained a great result using feature selection! Probably, too good to be true ...

Sanity check: randomly shuffle labels

In [ ]:
y_random = y.copy() 
np.random.shuffle(y_random)

In [ ]:
X_reduced = selector.fit_transform(X, y_random)
scores = cross_val_score(clf, X_reduced, y_random, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

### How to do a multistep analysis in a correct way? Use pipeline!

In [ ]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(SelectKBest(f_classif, n_features), 
                      LogisticRegressionCV())
scores = cross_val_score(pipe, X, y_random, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

scores = cross_val_score(pipe, X, y, scoring='accuracy', cv=5)
print('Accuracy of classification is {:0.2f} +- {:0.2f}'.format(np.mean(scores), np.std(scores)))

## Selection of hyperparameters & grid search 

In [ ]:
def f_poly(x, coefs):
    summands = [x**(power+1) * coef for power, coef in enumerate(coefs)]
    return np.array(summands).sum(0)

def get_function(coefs=None):
    if coefs is None:
        coefs = [1, -0.5, -1, 0.6]
    return lambda x: f_poly(x, coefs)

def get_dataset4(f, sample_size, noise_std=0.1):
    X = np.random.rand(sample_size, 1) * 2 - 1
    y = f(X)
    y += np.random.randn(*y.shape) * noise_std
    return X, y

def plot_dataset4(f, X=None, y=None, regr=None):
    X_plot = np.linspace(-1, 1, 100)[:, np.newaxis]
    plt.plot(X_plot, f(X_plot), label='True function')
    if X is not None and y is not None:
        plt.plot(X, y, '.r')
    if regr is not None:
        plt.plot(X_plot, regr.predict(X_plot), label='Prediction')
    plt.legend(loc='best')
    plt.ylim([-0.8, 0.6])

In [ ]:
f = get_function()
X, y = get_dataset4(f, 20)
X_test, y_test = get_dataset4(f, 100)
plot_dataset4(f, X, y)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
regr = make_pipeline(PolynomialFeatures(20), Ridge())
regr.fit(X, y)
plot_dataset4(f, X, y, regr)

In [ ]:
from sklearn.metrics.regression import mean_squared_error as mse

def get_errors(regr, X, y):
    y_predicted = regr.predict(X)
    mse(y, y_predicted)**0.5
    return mse(y, y_predicted)**0.5
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))

### Bias-variance tradeoff

<div style="width:100%; text-align:center">
<img src=http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png width=500px>
</div>
from a great tutorial http://scott.fortmann-roe.com/docs/BiasVariance.html

### We can use regularization to control model parameters

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(1e-9))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(1e9))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge(0.1))
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

### How to select model parameters?

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'ridge__alpha':10**np.linspace(-5, 5, 21)}
regr = make_pipeline(PolynomialFeatures(20), Ridge())
regr_grid = GridSearchCV(regr, parameters)
regr_grid.fit(X, y)

plot_dataset4(f, X, y, regr_grid)

### But what is we have more than one parameter to adjust?

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
parameters = {'learning_rate': [0.01, 0.025, 0.05, 0.1],
              'max_depth': [1, 2, 3, 4, 5],
              'n_estimators': [10, 20, 30, 50]
             }
clf_grid = GridSearchCV(clf, parameters, n_jobs=-1, verbose=True)
X, y = get_dataset3()
clf_grid.fit(X, y)

### An error with a fixed parameters is a random variable

In [ ]:
X, y = get_dataset4(f, 20)
regr = make_pipeline(PolynomialFeatures(20), Ridge())
regr.fit(X, y)
print('Root mean squared error is {}'.format(get_errors(regr, X_test, y_test)))
plot_dataset4(f, X, y, regr)

In [ ]:
def get_scores(regr, sample_size, n_repeats):
    scores = []
    for i in range(n_repeats):
        X, y = get_dataset4(f, sample_size)
        regr.fit(X, y)
        scores.append(get_errors(regr, X_test, y_test))
    return scores
regr = make_pipeline(PolynomialFeatures(20), Ridge())
scores = get_scores(regr, 20, 100)
sns.distplot(scores)

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(7, 7))
sns.despine(left=True)

def plot_hist(alpha, color, ax):
    regr = make_pipeline(PolynomialFeatures(20), Ridge(alpha=alpha))
    scores = get_scores(regr, 20, 100)
    sns.distplot(scores, color=color, ax=ax)
    ax.set_title('Regularization is {}'.format(alpha))

plot_hist(alpha=1e-9, color='r', ax=axes[0, 0])
plot_hist(alpha=1e-4, color='g', ax=axes[0, 1])
plot_hist(alpha=1e-1, color='b', ax=axes[1, 0])
plot_hist(alpha=1e9, color='m', ax=axes[1, 1])

## Take-away messages

 - Ideally, use an independent test set 
 - If you use multistep anysis always chain these steps into a single sklearn pipeline
 - As a sanity check, you can feed to your analysis random variables and compare the obrained results with the expected quality of random classification
 - Do not trust GridSearchCV results, always re-check the optimal comination of parameters